# Setup
Set ```rdSolverDir``` to ```"<YOUR_RECURDYN_INSTALL_DIR>\Bin\Solver\RDSolverRun.exe"``` in ```GlobalVariables.py```.

# Initialization

Initialize with imports and global variables


In [1]:
from recurdyn import *

# from recurdyn import Chart
# from recurdyn import MTT2D
# from recurdyn import FFlex
# from recurdyn import RFlex
from recurdyn import Tire
from datetime import datetime
import numpy as np
import re
import os
import glob
import os
import pandas as pd

from utils.modeling import *
from analysis.doe_batch import *
from analysis.doe_gui import *
from analysis.export_data import *

app = None
application = None
model_document = None
plot_document = None
model = None

ref_frame_1 = None
ref_frame_2 = None

#### Define `initialize()` and `dispose()` functions.


In [2]:
# initialize() should be called before ProcessNet function call.
def initialize():
    global app
    global application
    global model_document
    global plot_document
    global model

    app = dispatch_recurdyn()
    application = IApplication(app.RecurDynApplication)
    model_document = application.ActiveModelDocument
    if model_document is not None:
        model_document = IModelDocument(model_document)
    plot_document = application.ActivePlotDocument
    if plot_document is not None:
        plot_document = IPlotDocument(plot_document)

    if model_document is None and plot_document is None:
        application.PrintMessage("No model file")
        model_document = application.NewModelDocument("Examples")
    if model_document is not None:
        model_document = IModelDocument(model_document)
        model = ISubSystem(model_document.Model)

    return application, model_document, plot_document, model


# dispose() should be called after ProcessNet function call.


def dispose():
    global application
    global model_document

    model_document = application.ActiveModelDocument
    if model_document is not None:
        model_document = IModelDocument(model_document)
    else:
        return

    if not model_document.Validate():
        return
    # Redraw() and UpdateDatabaseWindow() can take more time in a heavy model.
    # model_document.Redraw()
    # model_document.PostProcess() # UpdateDatabaseWindow(), SetModified()
    # model_document.UpdateDatabaseWindow()
    # If you call SetModified(), Animation will be reset.
    # model_document.SetModified()
    model_document.SetUndoHistory("Python ProcessNet")

# Simulate using GUI solver

Call `analysis.doe_gui.RunDOE_GUI` with arguments.  
**You can modify DOE scenario by editing line 55~65 in `analysis/doe_gui.py`**  
This method is **_not parallelizable_**.

#### Arguments

- `ModelFileDir: str` Absolute path of model file (\*.rdyn).
- `TopFolderName: str` Folder name to create.
- `NumCPUCores: int` Number of CPU threads to use per simulation. Must be one of `[0,1,2,4,8,16]`
- `EndTime: float` Simulation end time.
- `NumSteps: int` Number of time steps.


In [3]:
RunDOE_GUI(
    ModelFileDir=f"{os.getcwd()}/SampleModel.rdyn",
    TopFolderName="TestDOE_GUI",
    NumCPUCores=8,
    EndTime=1,
    NumSteps=100,
)

RecurDyn V9R5 is not running.
Run RecurDyn
Analysis finished within 19.51sec.


#### Export data from results using `analysis.export_data.rplt2csv`
Exported variables are defined in ```GlobalVariables.GlobVar.DataExportTargets```.

In [4]:
rplt2csv(f"{os.getcwd()}/TestDOE_GUI")

Data exported(1/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0001.csv 
Data exported(2/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0002.csv 
Data exported(3/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0003.csv 
Data exported(4/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0004.csv 
Data exported(5/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0005.csv 
Data exported(6/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0006.csv 
Data exported(7/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0007.csv 
Data exported(8/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0008.csv 
Data exported(9/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0009.csv 
Data exported(10/10) d:\GitHub\ProcessNetPython-hb/TestDOE_GUI\TestDOE_GUI_0010.csv 
Export finished within 8.86sec.


# Simulate using batch solver

Call `analysis.doe_batch.RunDOE_Batch` with arguments.  
**You can control DOE scenario by editing line 59~67 in `analysis/doe_batch.py`**  
This method is **_parallelizable_**, but requires corresponding number of licenses.

#### Arguments

- `ModelFileDir: str` Absolute path of model file (\*.rdyn).
- `TopFolderName: str` Folder name to create.
- `NumCPUCores: int` Number of CPU threads to use per simulation. Must be one of `[0,1,2,4,8,16]`
- `EndTime: float` Simulation end time.
- `NumSteps: int` Number of time steps.
- `NumParallelBatches: int` Number of parallelized simulations (\*.bat) to create.
- `NumBatRunsOnThisPC: int` Number of simulations to run on your current machine. Defaults to `NumParallelBatches`. Range should be within `0`~`NumParallelBatches`.


In [5]:
RunDOE_Batch(
    ModelFileDir="SampleModel.rdyn",
    TopFolderName="TestDOE_Batch",
    NumCPUCores=8,
    EndTime=1,
    NumSteps=100,
    NumParallelBatches=5,
)

d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0001
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0002
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0003
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0004
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0005
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0006
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0007
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0008
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0009
d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_0010
Created batch executable d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_1.bat
Created batch executable d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_2.bat
Created batch executable d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_3.bat
Created batch executable d:\GitHub\ProcessNetPython-hb\TestDOE_Batch\TestDOE_Batch_4.bat
Created batch 

#### Export data from results using `analysis.export_data.rplt2csv`
Exported variables are defined in ```GlobalVariables.GlobVar.DataExportTargets```.

In [6]:
rplt2csv(f"{os.getcwd()}/TestDOE_Batch")

Data exported(1/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0001.csv 
Data exported(2/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0002.csv 
Data exported(3/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0003.csv 
Data exported(4/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0004.csv 
Data exported(5/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0005.csv 
Data exported(6/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0006.csv 
Data exported(7/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0007.csv 
Data exported(8/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0008.csv 
Data exported(9/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0009.csv 
Data exported(10/10) d:\GitHub\ProcessNetPython-hb/TestDOE_Batch\TestDOE_Batch_0010.csv 
Export finished within 9.32sec.
